In [ ]:
from langchain.graphs import Neo4jGraph
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os
from langchain.prompts import PromptTemplate
from langchain.chains import GraphCypherQAChain

load_dotenv()

NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "4NROB-HI"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
)

# Graph Transformer
# graph = Neo4jGraph(refresh_schema=False)

AZURE_API_KEY     = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_ENDPOINT    = os.getenv("AI_BASE")
AZURE_DEPLOYMENT  = os.getenv("AZURE_DEPLOYMENT")
AZURE_API_VERSION = os.getenv("AZURE_OPENAI_VERSION")


llm = AzureChatOpenAI(
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_ENDPOINT,
    deployment_name=AZURE_DEPLOYMENT,
    api_version=AZURE_API_VERSION,
    temperature=0,
    # max_tokens=2000,
)


In [4]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [12]:
from langchain_core.documents import Document


text = """

L1	L2	L3	L4	001-AS	133-FES	008-SWARAJ	002-HO	Powerol	CE	Agri	004-MDS	Synergy	007-SBU	TW	134-TPDS	135-INTNL OPER	138-CONT SOUR	140-NON MRV	141-SOURCING	139-CORP
Inventories				53671516382	22944652153	9009308396	0	1325324816	0	0	0	0	101756237.5	1760985713	0	0	0	0	0	0
	Raw Material & Mfg. Componets			36468349988	8158078317	1466552014	0	864491092.4	0	0	0	0	0	1464491878	0	0	0	0	0	0
		Raw Material & Mfg. Componet		33001594819	7779327669	1309178305	0	864491092.4	0	0	0	0	0	1464491878	0	0	0	0	0	0
			Closing Stock- Gunny Bags	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Clg Stock - Raw Material-B/s	30235079480	6542914168	1270706601	0	294446899.7	0	0	0	0	0	1464491878	0	0	0	0	0	0
			Clg Stock - Mfg Component B/s	2766515339	1236413500	38471704	0	570044192.7	0	0	0	0	0	0	0	0	0	0	0	0
		Stock In Transit - Raw Material		3466755169	378750648	157373709.4	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Clg Stock In Transit - Rm - B/s	3466755169	378750648	157373709.4	0	0	0	0	0	0	0	0	0	0	0	0	0	0
	Work-in-Progress			749578581.5	193331922.2	508730464	0	0	0	0	0	0	0	135699393.6	0	0	0	0	0	0
			WIP- Seed Potato Facility A/c	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Construction Work In Progress	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Clg Stock - Wip - B/s	749578581.5	193331922.2	508730464	0	0	0	0	0	0	0	135699393.6	0	0	0	0	0	0
			Clg Stock - Construction W I P - B/s	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
	Finished Goods			10435364997	11506612938	6260702023	0	175333864.3	0	0	0	0	0	122817402.2	0	0	0	0	0	0
		Stock In Transit - Finised Goods		10435364997	11506612938	6260702023	0	175333864.3	0	0	0	0	0	122817402.2	0	0	0	0	0	0
			Clg Stock In Transit - FG B/s	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
		Finished Goods		0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Clg Stock - Finished Goods - B/s	10435364997	11506612938	6260702023	0	175333864.3	0	0	0	0	0	122817402.2	0	0	0	0	0	0
	Stock in Trade			4143497278	2482314051	561774256	0	285499859.6	0	0	0	0	0	0	0	0	0	0	0	0
		Stock in Trade		4143497278	2482314051	561774256	0	285499859.6	0	0	0	0	0	0	0	0	0	0	0	0
			Stock in trade	4143497278	2482314051	561774256	0	285499859.6	0	0	0	0	0	0	0	0	0	0	0	0
		Stock In Transit - Traded Goods.............		0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Stock in transit - Traded goods	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
	Stores and Spares			604815070.6	99888608.63	20602986.79	0	0	0	0	0	0	96520811.48	4557040.9	0	0	0	0	0	0
			Inventory Oils, Grease & Chemicals	62117423.19	59355378.47	10436570.32	0	22224.3	0	0	0	0	0	3621117.71	0	0	0	0	0	0
			Inventorie Consumable Stores & Misc.	93694887.81	40284396.67	13938464.01	0	0	0	0	0	0	0	9035547.28	0	0	0	0	0	0
			G-Stock Materiali Ricoperti CBN	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Stock in transit - Stores & Spares	0	763263	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Inventory Stores- Manual Adj	443645826.3	-1713583.34	-5714141.25	0	-22224.3	0	0	0	0	0	-9985961.43	0	0	0	0	0	0
			Inventory Packing Material	5356933.38	1199153.83	1942093.71	0	0	0	0	0	0	96520811.48	1886337.34	0	0	0	0	0	0
	Inventory Machinery Spares			807477663.2	238067319	153132376.7	0	0	0	0	0	0	5235426.06	15255230.68	0	0	0	0	0	0
			Inventory Spares	801894547.7	240085712.6	238801300.5	0	0	0	0	0	0	235426.06	15201315.93	0	0	0	0	0	0
			Inventory Machinery Spares-Imported	15267299.48	4225495.81	0	0	0	0	0	0	0	0	53914.75	0	0	0	0	0	0
			Inventory Spares- Manual Adj	-9684183.92	-6243889.46	-85668923.76	0	0	0	0	0	0	5000000	0	0	0	0	0	0	0
			In Transit - Stores & Spares	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-
	Tools			462432803	266358996.8	37814275.65	0	0	0	0	0	0	0	18164767.89	0	0	0	0	0	0
			Inventory Tools	454986377.1	328455020.6	77200421	0	0	0	0	0	0	0	19019802.89	0	0	0	0	0	0
			Inventory Manufactured Tools	92781700.65	13554703.59	459718.8	0	0	0	0	0	0	0	235194.58	0	0	0	0	0	0
			Inventory Tools- Manual Adj	-85355475.06	-75650727.32	-39845864.15	0	0	0	0	0	0	0	-1090229.58	0	0	0	0	0	0
			Stock in transit - Tools	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Inventory Dies	20200.36	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
	Other Inventory			0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Food & Beverages	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Ind-AS  Inventories	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
			Plant & Machinery Held For Sale	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0


"""    

# wrap in a Document first
document = Document(page_content=text)

# transform into graph documents
graph_documents = await llm_transformer.aconvert_to_graph_documents([document])

In [ ]:
print(f"Nodes:{graph_documents[0].nodes[0]}")    
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:id='Inventories' type='Inventory' properties={}
Relationships:[Relationship(source=Node(id='Inventories', type='Inventory', properties={}), target=Node(id='Raw Material & Mfg. Components', type='Inventory', properties={}), type='CONTAINS', properties={}), Relationship(source=Node(id='Inventories', type='Inventory', properties={}), target=Node(id='Work-In-Progress', type='Inventory', properties={}), type='CONTAINS', properties={}), Relationship(source=Node(id='Inventories', type='Inventory', properties={}), target=Node(id='Finished Goods', type='Inventory', properties={}), type='CONTAINS', properties={}), Relationship(source=Node(id='Inventories', type='Inventory', properties={}), target=Node(id='Stock In Trade', type='Inventory', properties={}), type='CONTAINS', properties={}), Relationship(source=Node(id='Inventories', type='Inventory', properties={}), target=Node(id='Stores And Spares', type='Inventory', properties={}), type='CONTAINS', properties={}), Relationship(source=Node(